In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.request, json
import pickle


In [ ]:
latlonglist = pd.read_csv('counts_with_latlong.csv', sep='\t', index_col=0)

In [ ]:
#latitude_coords = latlonglist['GTFS Latitude']
latlonglist_nonempty = latlonglist.dropna(subset=['GTFS Latitude', 'GTFS Longitude'] )

In [ ]:
set(latlonglist_nonempty['GTFS Longitude'].isnull())

In [ ]:
latlonglist_nonempty.reset_index(inplace=True)

In [ ]:
FCC_urls = []


In [ ]:
for i in range(0, len(latlonglist_nonempty['STATION'])):
    FCC_urls.append('https://geo.fcc.gov/api/census/block/find?latitude={}&longitude={}&format=json'.format(latlonglist_nonempty['GTFS Latitude'][i], latlonglist_nonempty['GTFS Longitude'][i]))


In [ ]:
geoid = []

In [ ]:
for i in FCC_urls:
    with urllib.request.urlopen(i) as url:
        data = json.loads(url.read().decode())
        geoid.append(data['Block']['FIPS'])


In [ ]:
latlonglist_nonempty['GEOID10'] = [int(x[0:11]) for x in geoid]

In [ ]:
census_female_pop = pd.read_csv('2010_FEMALE_POPULATION2.csv', sep=',', header=0)

In [ ]:
totalcounts_merged_femalepop = pd.merge(latlonglist_nonempty, census_female_pop, on='GEOID10', how='left')
                                                

In [ ]:
with open('pop_merged', 'wb') as to_write:
    pickle.dump(totalcounts_merged_femalepop, to_write)

In [2]:
with open('pop_merged', 'rb') as read_file:
    pop_merged = pickle.load(read_file)

In [4]:
more_census_data = pd.read_csv('2010_more_demogs.csv', sep=',')

In [6]:
more_census_data.drop(labels='2010_Female_Population_(US_Census)', axis=1, inplace=True)

In [12]:
census_merged_counts = pd.merge(pop_merged, more_census_data, on='GEOID10', how='left')



In [14]:
with open('census_merged_counts', 'wb') as to_write:
    pickle.dump(census_merged_counts, to_write)

    